## Problem 1: Hand-crafted Network
### 1.1: Hand-crafted Neurons
#### 1.1.1: Logical OR

We define the logical OR as:

$$
f(\mathbf{z}) = \text{sign}(\mathbf{z} \cdot \mathbf{\mathbb{1}_{D,1}})
$$

Therefore we have that $b = 0, \ \mathbf{w} = \mathbf{\mathbb{1}_{D,1}}, \ \phi(x) = \text{sign}(x)$.

#### 1.1.2: Masked OR

We define the masked OR as:

$$
g(\mathbf{z}; \mathbf{c}) = \text{sign}(\mathbf{z} \cdot \mathbf{c})
$$

Therefore $b = 0, \ \mathbf{w} = \mathbf{c}, \ \phi(x) = |\text{sign}(x)|$.

#### 1.1.3: Perfect Match

We define the perfect match neuron as:

$$
h(\mathbf{z}; \mathbf{c}) = 1 - |\text{sign}((\mathbf{z} - \mathbf{c}) \cdot \mathbf{\mathbb{1}_{D,1}})| 
$$

Therefore $b = 0, \ \mathbf{w} = \mathbf{\mathbb{1}_{D,1}}, \ \phi(x) = 1 - |\text{sign}(x)|$

## Problem 2: Linear Activation Function

Since $\phi_l$ is the identity function we can define the activation of each layer as:

$$\mathbf{Z_l} = \mathbf{Z_{l-1}} \cdot \mathbf{B_l} + \mathbf{b_l}$$ 


Therefore the activation $\mathbf{Z_L}$ of the last layer $L$ can be written as follows:

$$ \mathbf{Z_L} = (\dots(\mathbf{X} \mathbf{B_1} + \mathbf{b_1}) \mathbf{B_2} + \mathbf{b_3})\dots) \mathbf{B_L} + \mathbf{b_L} $$

This form can be reduced using the following pairwise reduction for some activations in layers $l$ and $l+1$ until there is only one layer left:

$$
\begin{align}
\mathbf{Z_{l+1}} &= (\mathbf{Z_{l-1}} \mathbf{B_l} + \mathbf{b_l}) \mathbf{B_{l+1}} + \mathbf{b_{l+1}} \\
&= \mathbf{Z_{l-1}} \mathbf{B_l} \mathbf{B_{l+1}} + \mathbf{b_l} \mathbf{B_{l+1}} + \mathbf{b_{l+1}} \\
&= \mathbf{Z_{l-1}} \bar{\mathbf{B_l}} + \bar{\mathbf{b_{l}}}
\end{align}
$$

The new parameters are defined as $\bar{\mathbf{B_l}} = \mathbf{B_l} \mathbf{B_{l+1}}$ and $\bar{\mathbf{b_l}} = \mathbf{b_l} \mathbf{B_{l+1}} + \mathbf{b_{l+1}}$.

## Problem 3

<div style="color: green; font-weight: bold">The forward function is essentially the same as in the sample solution. <br>
    The backwards path is also identical, but with one more multiplication. Performance wise the sample solution should be better. 
</div>

In [69]:
import numpy as np
from sklearn import datasets
# ###################################
class ReLULayer ( object ):
    def forward (self , input ):
        # remember the input for later backpropagation
        self.input = input
        # return the ReLU of the inpu't
        # basic relu implementation
        relu = np.maximum(0, input)
        return relu
    def backward (self , upstream_gradient ) :
        # compute the derivative of ReLU from upstream_gradient and the stored input
        # look for which input relu was called. if the input was <= 0, relu was not aplied, and hence derivative needs also be zero
        # if input > 0, we need to set the derivative to 1
        derivative = np.where(self.input > 0, 1, 0)
        downstream_gradient = upstream_gradient * derivative
        return downstream_gradient
    def update (self , learning_rate ):
        pass # ReLU is parameter - free

<div style="color: green; font-weight: bold">forward is essentially the same, however, the nummerator is calculated differently in the sample solution, but it is stated that this difference does not impact the result <br>
For the backwards path I certainly mixed some things up. However the saoftmax derivative calculation is almost correct, but it used the inputs as values instead of the predicted posteriors. Also I forgot to divide each sample by the length of the batch. Furthermore this solution here uses many for loops and should be rewritten in a vectorized manner as in the sample solution</div>

In [70]:
class OutputLayer( object ):
    def __init__ (self , n_classes ):
        self.n_classes = n_classes
    def forward (self, input ):
        # remember the input for later backpropagation
        self.input = input
        # return the softmax of the input
        softmax = np.exp(input) / np.sum(np.exp(input))
        return softmax
    def backward(self , predicted_posteriors , true_labels ) :
        # return the loss derivative with respect to the stored inputs
        # (use cross - entropy loss and the chain rule for softmax ,
        # as derived in the lecture )
        
        #calculate cross entropy loss derivative
        loss_derivative = []
        for row,label in enumerate(true_labels):
            sample_loss = []
            for class_idx in range(2):
                if class_idx == label:
                    sample_loss.append(-1/predicted_posteriors[row][class_idx])
                else:
                    sample_loss.append(0)
            loss_derivative.append(sample_loss)        
            
        # calculate softmaxe derivative of the inputs 
        softmax_derivative = []
        for row,label in enumerate(true_labels):
            sample_softmax_derivative = []
            for class_idx in range(2):
                if class_idx == label:
                    sample_softmax_derivative.append(predicted_posteriors[row][class_idx]-1)
                else:
                    sample_softmax_derivative.append(predicted_posteriors[row][class_idx])
            softmax_derivative.append(sample_softmax_derivative)
        # mltiply
        softmax_derivative = np.asarray(softmax_derivative)
        loss_derivative = np.asarray(loss_derivative)
        # I do think i need to multiply the derivatives and I also believe that i implemented them according to the lecture.
        # However if I multiply them, i receive a 2 x 2 matrix which is incompatible with the next gradient to compute...
        # Either this is here is false or the LinearLayer gradient calculation
        softmax_derivative /= len(true_labels) # @ loss_derivative
        return softmax_derivative
    def update(self , learning_rate ):
        pass # softmax is parameter - free

<div style="color: green; font-weight: bold">Initialization is quite similar, with more adjustments to the np.random function in the sample solution, which shouldn't make that big of a difference. <br>
The forward path, it is practically the same. The only idfference is matmul instead of np.dot, which is however equivallent for 2D-Arrays.<br>
The backwards path is identical to the sample solution</div>

In [71]:
class LinearLayer( object ):
    def __init__ (self , n_inputs , n_outputs ):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        # randomly initialize weights and intercepts for the feature dimensions
        self.B = np.random.normal(size=(n_inputs, n_outputs)) 
        self.b = np.random.normal(size=(n_outputs))
    def forward(self , input ):
        # remember the input for later backpropagation
        self.input = np.asarray(input)
        # compute the scalar product of input and weights
        # ( these are the preactivations for the subsequent non - linear layer )
        # the linear preactivation claclulation of the weights multiplied by the inputs and added the biases
        preactivations = np.matmul(self.input, self.B) + self.b 
        return preactivations
    def backward(self , upstream_gradient ) :
        # compute the derivative of the weights from
        # upstream_gradient and the stored input
        self.grad_b = np.sum(upstream_gradient, axis=0)
        self.grad_B = np.matmul(self.input.T, upstream_gradient)
        # compute the downstream gradient to be passed to the preceding layer
        downstream_gradient = np.matmul(upstream_gradient, self.B.T)
        return downstream_gradient
    def update(self , learning_rate ):
        # update the weights by batch gradient descent
        self.B = self.B - learning_rate * self.grad_B
        self.b = self.b - learning_rate * self.grad_b


<div style="color: green; font-weight: bold">The backward function is computed the same way as in the sample <br>
</div>

In [72]:
class MLP( object ):
    def __init__ (self , n_features , layer_sizes ):
        # constuct a multi - layer perceptron
        # with ReLU activation in the hidden layers and softmax output
        # (i.e. it predicts the posterior probability of a classification problem )
        # n_features : number of inputs
        # len ( layer_size ): number of layers
        # layer_size [k]: number of neurons in layer k
        # ( specifically : layer_sizes [ -1] is the number of classes )
        self.n_layers = len( layer_sizes )
        self.layers = []
        # create interior layers ( linear + ReLU )
        n_in = n_features
        for n_out in layer_sizes[:-1]:
            self.layers.append( LinearLayer(n_in , n_out ))
            self.layers.append( ReLULayer() )
            n_in = n_out
        # create last linear layer + output layer
        n_out = layer_sizes[ -1]
        self.layers.append( LinearLayer(n_in , n_out ))
        self.layers.append( OutputLayer( n_out ))
    def forward (self , X):
        # X is a mini - batch of instances
        batch_size = X.shape[0]
        # flatten the other dimensions of X (in case instances are images )
        X = X.reshape( batch_size , -1)
        # compute the forward pass
        # ( implicitly stores internal activations for later backpropagation )
        result = X
        for layer in self.layers:
            result = layer.forward( result )
        return result
    def backward(self , predicted_posteriors , true_classes ):
        # init result of the last layer
        result = self.layers[-1].backward(predicted_posteriors , true_classes)
        for layer in reversed(self.layers[:-1]):
            # set the result to the according backpropagation function
            result = layer.backward(result)
        return result
    def update(self , X, Y, learning_rate ):
        posteriors = self.forward(X)
        self.backward(posteriors,Y)
        for layer in self.layers :
            layer.update( learning_rate )
    def train(self , x, y, n_epochs , batch_size , learning_rate ):
        N = len (x)
        n_batches = N // batch_size
        for i in range( 2 ):
            print (" Epoch ", i)
            # reorder data for every epoch
            # (i.e. sample mini - batches without replacement )
            permutation = np. random.permutation(N)
            for batch in range( n_batches ):
                # create mini - batch
                start = batch * batch_size
                x_batch = x[ permutation [ start:start + batch_size ]]
                y_batch = y[ permutation [ start:start + batch_size ]]
                # perform one forward and backward pass and update network parameters
                self.update( x_batch , y_batch , learning_rate )

<div style="color: green; font-weight: bold">The idea f calculating the error rate was  the same as in the sample solution. The sample solution used vectorization which is much faster and does not need the for loop that is used here. However this vcersion here is not correctly implemented as it outputs undesired results for the error rate because of the usage of np.where
</div>

In [73]:
# set training / test set size
N = 2000
# create training and test data
X_train , Y_train = datasets.make_moons (N, noise =0.05)
X_test , Y_test = datasets.make_moons (N, noise =0.05)
n_features = 2
n_classes = 2
# standardize features to be in [ -1 , 1]
offset = X_train.min(axis =0)
scaling = X_train.max(axis =0) - offset
X_train = (( X_train - offset ) / scaling - 0.5) * 2.0
X_test = (( X_test - offset ) / scaling - 0.5) * 2.0
# set hyperparameters ( play with these !)
layer_sizes = [5 , 5, n_classes ]
n_epochs = 5
batch_size = 200
learning_rate = 0.05
# create network
network = MLP( n_features , layer_sizes )
# train
network.train( X_train , Y_train , n_epochs , batch_size , learning_rate )
# test
predicted_posteriors = network.forward( X_test )
print(f"Predicted Posteriors: {predicted_posteriors}")
# determine class predictions from posteriors by winner -takes - all rule

predicted_classes = [ np.argmax(sample) for sample in predicted_posteriors]
# compute and output the error rate of predicted_classes
error_rate = (1 - (len(np.where(Y_test == predicted_classes))) / len(Y_test))
# compute and output the error rate of predicted_classes
print (" error rate :", error_rate )

 Epoch  0
 Epoch  1
Predicted Posteriors: [[1.93695100e-05 1.33823869e-03]
 [1.66849034e-06 6.91362725e-05]
 [7.52671716e-06 4.69552354e-04]
 ...
 [8.03818829e-06 5.01481372e-04]
 [7.52253506e-06 4.58712289e-04]
 [6.12584400e-06 3.54084504e-04]]
 error rate : 0.9995


In [74]:
# test the implementation
test_cases = [  MLP( n_features , [2 , 2 , n_classes ]),
                MLP( n_features , [3 , 3 , n_classes ]),
                MLP( n_features , [5 , 5 , n_classes ]),
                MLP( n_features , [30 , 30 , n_classes ])]

for test_nn in test_cases:
    print(50*"-")
    test_nn.train(X_train , Y_train , n_epochs , batch_size , learning_rate)
    predicted_posteriors = test_nn.forward( X_test )
    print(f"Predicted Posteriors: {predicted_posteriors}")
    predicted_classes = [ np.argmax(sample) for sample in predicted_posteriors]
    error_rate = 1 - (len(np.where(Y_test == predicted_classes)) / len(Y_test))
    print (" error rate :", error_rate )
# the test results are bad because of the wrong implementation of the backpropagation in the outputlayer.

--------------------------------------------------
 Epoch  0
 Epoch  1
Predicted Posteriors: [[8.40941617e-05 4.15905838e-04]
 [8.40941617e-05 4.15905838e-04]
 [8.40941617e-05 4.15905838e-04]
 ...
 [8.40941617e-05 4.15905838e-04]
 [8.40941617e-05 4.15905838e-04]
 [8.40941617e-05 4.15905838e-04]]
 error rate : 0.9995
--------------------------------------------------
 Epoch  0
 Epoch  1
Predicted Posteriors: [[8.77625077e-05 5.69352206e-04]
 [6.40572368e-05 3.90528277e-04]
 [1.06201825e-04 7.15402119e-04]
 ...
 [5.40236897e-05 3.18468708e-04]
 [4.78781144e-05 2.75593125e-04]
 [4.98587099e-05 2.89298893e-04]]
 error rate : 0.9995
--------------------------------------------------
 Epoch  0
 Epoch  1
Predicted Posteriors: [[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]
 error rate : 0.9995
--------------------------------------------------
 Epoch  0
 Epoch  1
Predicted Posteriors: [[nan nan]
 [nan nan]
 [nan nan]
 ...
 [nan nan]
 [nan nan]
 [nan nan]]
 error rate :

/tmp/ipykernel_2040/2305622641.py:8: RuntimeWarning: overflow encountered in exp
  softmax = np.exp(input) / np.sum(np.exp(input))
/tmp/ipykernel_2040/2305622641.py:8: RuntimeWarning: invalid value encountered in divide
  softmax = np.exp(input) / np.sum(np.exp(input))
/tmp/ipykernel_2040/2305622641.py:21: RuntimeWarning: divide by zero encountered in scalar divide
  sample_loss.append(-1/predicted_posteriors[row][class_idx])
